<center> 
    <a href="https://github.com/nebuly-ai/nebullvm#how-nebullvm-works" target="_blank" style="text-decoration: none;"> How Nebullvm Works </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#tutorials" target="_blank" style="text-decoration: none;"> Tutorials </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#benchmarks" target="_blank" style="text-decoration: none;"> Benchmarks </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#installation" target="_blank" style="text-decoration: none;"> Installation </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#get-started" target="_blank" style="text-decoration: none;"> Get Started </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#optimization-examples" target="_blank" style="text-decoration: none;"> Optimization Examples </a>
</center>
<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Discord </a> |
    <a href="https://nebuly.ai/" target="_blank" style="text-decoration: none;"> Website </a> |
    <a href="https://www.linkedin.com/company/72460022/" target="_blank" style="text-decoration: none;"> LinkedIn </a> |
    <a href="https://twitter.com/nebuly_ai" target="_blank" style="text-decoration: none;"> Twitter </a>
</center>

# Accelerate PyTorch YOLO with nebullvm

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using the open-source library nebullvm.

With nebullvm's latest API, you can speed up models up to 10 times without any loss of accuracy (option A), or accelerate them up to 20-30 times by setting a self-defined amount of accuracy/precision that you are willing to trade off to get even lower response time (option B). To accelerate your model, nebullvm takes advantage of various optimization techniques such as deep learning compilers (in both option A and option B), quantization, half accuracy, and so on (option B).

Let's jump to the code.

# Install and test YOLO

Let's install YOLO.

In [ ]:
! pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

We start by downloading the model from the Torch hub and running an initial inference on an image.

In [ ]:
import copy
import time
import types

import torch

In [ ]:
# Load Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, force_reload=True)

# Images
imgs = ['https://ultralytics.com/images/zidane.jpg']  # batch of images

Let's now calculate the time required to run a prediction as an average over 100 tests.

In [ ]:
times = []
for _ in range(100):
    starting_time = time.time()
    # Inference
    results = model(imgs)
    times.append((time.time()-starting_time)*1000)
yolo_vanilla_time = sum(times) / len(times)
print(f"Average prediction time: {yolo_vanilla_time} ms")

In [ ]:
type(model)

In [ ]:
#results.print()
results.show()

Here we are! We got a good prediction, but it took a while :) Let's see if we are able to speed up the model a little bit without losing in performance.

## Optimization with nebullvm

In [ ]:
from nebullvm import optimize_torch_model

First, we need to slightly modify YOLO's forward method. 

The last layer of the YOLOv5 implementation can create problems on certain hardware for some deep learning compilers that run on the nebullvm core. Since nebullvm aims to be hardware agnostic, we circumvent any potential obstacles by splitting the network body from the head (the last layer) and optimizing only the body.

In [ ]:
core_model = copy.deepcopy(model.model.model)

In [ ]:
def _forward_once(self, x, profile=False, visualize=False):
    y, dt = [], []  # outputs
    for m in self.model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
        if profile:
            self._profile_one_layer(m, x, dt)
        x = m(x)  # run
        y.append(x if m.i in self.save else None)  # save output
        if visualize:
            feature_visualization(x, m.type, m.i, save_dir=visualize)
    self.last_y = y
    return x
core_model._forward_once = types.MethodType(_forward_once, core_model)

The reimplementation of the forward method is needed since we need to store the ys for giving to the head the right tensors as input.

In [ ]:
class CoreModelWrapper(torch.nn.Module):
    def __init__(self, core_model, output_idxs):
        super().__init__()
        self.core = core_model
        self.idxs = output_idxs
        
    def forward(self, *args, **kwargs):
        x = self.core(*args, **kwargs)
        return tuple(x if j == -1 else self.core.last_y[j] for j in self.idxs)

In [ ]:
list_of_layers = list(core_model.model.children())
last_layer = list_of_layers.pop(-1)

core_model.model = torch.nn.Sequential(*list_of_layers)
core_wrapper = CoreModelWrapper(core_model, last_layer.f)

Now we are ready for optimizing the body of YOLOv5 using the `nebullvm` function `optimize_torch_model`.

Nebullvm was built to be very easy to use. To optimize a model, you only need to specify the model, the batch size and input size for each input tensor, and a directory in which to save the optimized model. In the example, we chose the same directory in which this notebook runs.

With the latest API, there are two ways to use nebullvm:

- Option A: Accelerate the model up to ~10 times without losing in performances (accuracy/precision/etc.)
- Option B: Accelerate the model up to ~30 times with a pre-defined maximum loss in performances
    - B1: Performance is measured on your dataset with a performance metric that you specify.
    - B2: Performance is measured on your dataset using precision as a metric.
    - B3: Performance is measured using precision as a metric.
    
To learn more about how to use nebullvm, check out the <a href="https://github.com/nebuly-ai/nebullvm#get-started" target="_blank" style="text-decoration: none;"> readme on GitHub </a>.

In this example, we provide the code to run option B.2 model and you find commented out the code for option A.

In [ ]:
## Option A: 2-10x acceleration, NO performance loss
# model_optimized = optimize_torch_model(
#     model=core_wrapper,
#     batch_size=1,
#     input_sizes=[(3, 384, 640)],
#     save_dir=".",
#     use_torch_api=True,
# )

In [ ]:
from PIL import Image
import requests
import numpy as np

In [ ]:
img_name = "zidane.png"
Image.open(requests.get(imgs[0], stream=True).raw).save(img_name)

In [ ]:
def read_and_crop(im, original_model, img_size):
    p  =  next(original_model.parameters())
    im = Image.open(requests.get(im, stream=True).raw if str(im).startswith('http') else im)
    max_y, max_x = im.size
    ptr_x = np.random.choice(max_x-img_size[0])
    ptr_y = np.random.choice(max_y-img_size[1])
    im = np.array(im.crop((ptr_y, ptr_x, ptr_y + img_size[1], ptr_x + img_size[0])))
    x = np.expand_dims(im, axis=0)
    x = np.ascontiguousarray(np.array(x).transpose((0, 3, 1, 2)))  # stack and BHWC to BCHW
    x = torch.from_numpy(x).to(p.device).type_as(p) / 255  # uint8 to fp16/32
    return x

In [ ]:
input_data = [((read_and_crop(img_name, core_model, (384, 640)),), None) for _ in range(500)]

In [ ]:
# Option B.2: 2-30x acceleration, maximum performance loss below your 
# perf_loss_ths, where performance is measured on your dataset dl_nebullvm
# using precision as a metric. Read more on GitHub on how to set perf_loss_ths.

model_optimized = optimize_torch_model(
    model=core_wrapper,
    save_dir=".",
    dataloader=input_data,
    use_torch_api=True,
    perf_loss_ths=3,
)

Now let's regroup together the optimized body and the head of YOLO.

In [ ]:
class OptimizedYolo(torch.nn.Module):
    def __init__(self, optimized_core, head_layer):
        super().__init__()
        self.core = optimized_core
        self.head = head_layer
    
    def forward(self, x, *args, **kwargs):
        x = list(self.core(x)) # it's a tuple
        return self.head(x)

In [ ]:
final_core = OptimizedYolo(model_optimized, last_layer)

In [ ]:
model.model.model = final_core

Finally we can check the speedup. Let's calculate the time required to run a prediction as an average over 100 tests.

In [ ]:
times = []
for _ in range(100):
    st = time.time()
    results = model(imgs)
    times.append((time.time() - st)*1000)
yolo_optimized_time = sum(times) / len(times)
print(f"Average prediction time: {yolo_optimized_time} ms")

In [ ]:
results.show()

What an amazing result, right?!? Stay tuned for more cool content from the Nebuly team :) 

## Print the optimization results

In [ ]:
# Enter here your username
your_username = "username"

In [ ]:
## Uncomment the following line for installing gputil if you are running on an NVIDIA GPU.
#!pip install gputil

In [ ]:
import cpuinfo
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
cpu_info = cpuinfo.get_cpu_info()['brand_raw']
gpu_info = "no"
if torch.cuda.is_available():
    import GPUtil
    gpus = GPUtil.getGPUs()
    gpu_info = list(gpus)[0].name

In [ ]:
message = f"""
Hello, I'm {your_username}!
I've tested nebullvm on the following setup:

Hardware: {cpu_info} CPU and {gpu_info} GPU.
Model: YOLOv5s
Vanilla performance: {round(yolo_vanilla_time, 2)}ms
Optimized performance: {round(yolo_optimized_time, 2)}ms
Speedup: {round(yolo_vanilla_time/yolo_optimized_time, 1)}x
"""
print(message)

Great! Was it easy? How are the results? Do you have any comments? Share your optimization results and thoughts with our community on Discord, where we chat about nebullvm and AI acceleration.

Note that the acceleration of nebullvm depends very much on the hardware configuration and your AI model. Given the same input model, nebullvm can accelerate it by 10 times on some machines and perform poorly on others.

If you want to learn more about how nebullvm works, look at other tutorials and performance benchmarks, check out the links below or write to us on Discord.

<center> 
    <a href="https://github.com/nebuly-ai/nebullvm#how-nebullvm-works" target="_blank" style="text-decoration: none;"> How Nebullvm Works </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#tutorials" target="_blank" style="text-decoration: none;"> Tutorials </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#benchmarks" target="_blank" style="text-decoration: none;"> Benchmarks </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#installation" target="_blank" style="text-decoration: none;"> Installation </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#get-started" target="_blank" style="text-decoration: none;"> Get Started </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#optimization-examples" target="_blank" style="text-decoration: none;"> Optimization Examples </a>
</center>
<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Discord </a> |
    <a href="https://nebuly.ai/" target="_blank" style="text-decoration: none;"> Website </a> |
    <a href="https://www.linkedin.com/company/72460022/" target="_blank" style="text-decoration: none;"> LinkedIn </a> |
    <a href="https://twitter.com/nebuly_ai" target="_blank" style="text-decoration: none;"> Twitter </a>
</center>